In [4]:
import sqlite3
import requests
from PIL import Image
import io
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Caminho do banco de dados
DB_PATH = "document_comparisons.db"

# Função para inicializar a tabela Candidatas
def initialize_candidates_table():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Candidatas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        doc1_id TEXT NOT NULL,
        doc2_id TEXT NOT NULL,
        similarity_score FLOAT NOT NULL,
        comparison_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(doc1_id, doc2_id)
    )
    """)
    conn.commit()
    conn.close()

# Função para buscar documentos da API
def search_documents(collection=None, max_results=10, start=0):
    """
    Busca documentos da API.
    """
    base_url = "https://metadata.idl.ucsf.edu/solr/ltdl3/query"
    query = f"collection:{collection}" if collection else "*:*"
    params = {
        "q": query,
        "wt": "json",
        "rows": max_results,
        "start": start
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json().get("response", {}).get("docs", [])
    else:
        raise ValueError(f"Erro na API: {response.status_code}")

# Função para gerar URL do arquivo TIF
def get_tif_url(doc_id):
    return f"https://download.industrydocuments.ucsf.edu/{doc_id[0]}/{doc_id[1]}/{doc_id[2]}/{doc_id[3]}/{doc_id}/{doc_id}.tif"

# Função para carregar documento TIF
def load_tif(doc_id):
    tif_url = get_tif_url(doc_id)
    response = requests.get(tif_url, stream=True)
    if response.status_code == 200:
        img = Image.open(io.BytesIO(response.content))
        return img.copy()  # Retorna apenas a primeira página
    else:
        raise ValueError(f"Erro ao carregar o arquivo TIF: {response.status_code}")

# Função para comparar imagens
def compare_pil_images(img1, img2):
    img1_gray = np.array(img1.convert("L"))
    img2_gray = np.array(img2.convert("L"))

    if img1_gray.shape != img2_gray.shape:
        img2_gray = np.array(img2.resize(img1.size).convert("L"))

    similarity_index, _ = ssim(img1_gray, img2_gray, full=True)
    return similarity_index

# Função para salvar uma comparação candidata
def save_candidate(doc1_id, doc2_id, similarity_score):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    try:
        cursor.execute("""
        INSERT INTO Candidatas (doc1_id, doc2_id, similarity_score)
        VALUES (?, ?, ?)
        """, (doc1_id, doc2_id, similarity_score))
    except sqlite3.IntegrityError:
        pass  # Evita duplicatas

    conn.commit()
    conn.close()

# Função para contar as linhas da tabela
def count_candidates():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("SELECT COUNT(*) FROM Candidatas")
    count = cursor.fetchone()[0]

    conn.close()
    return count

# Processo de busca e alimentação da tabela
def search_and_feed_candidates(collection=None, max_results=10):
    initialize_candidates_table()

    start = 0  # Início da busca na API

    while count_candidates() < 1000:
        # Buscar documentos da API
        try:
            documents = search_documents(collection=collection, max_results=max_results, start=start)
        except ValueError as e:
            print(e)
            break

        # Realizar comparações entre pares de documentos
        for i, doc1 in enumerate(documents):
            for doc2 in documents[i+1:]:
                try:
                    doc1_id = doc1.get("id")
                    doc2_id = doc2.get("id")

                    # Carregar imagens dos documentos
                    img1 = load_tif(doc1_id)
                    img2 = load_tif(doc2_id)

                    # Comparar as imagens
                    similarity_score = compare_pil_images(img1, img2)

                    # Salvar no banco se a similaridade for maior que 0.86
                    if similarity_score > 0.85:
                        save_candidate(doc1_id, doc2_id, similarity_score)

                        print(f"Gravado: {doc1_id} vs {doc2_id} - Similaridade: {similarity_score:.2f}")

                except Exception as e:
                    print(f"Erro na comparação: {e}")

                # Checar se alcançamos 1000 candidatos
                if count_candidates() >= 1000:
                    print("Tabela de candidatos preenchida com 1000 comparações.")
                    return

        start += max_results  # Avançar para a próxima página de resultados

In [5]:
def drop_candidates_table():
    """
    Remove a tabela Candidatas do banco de dados.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS Candidatas")
    conn.commit()
    conn.close()

In [26]:
drop_candidates_table()

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('one_page_document_comparisons.csv')

In [31]:
df = df[df['same_pattern'] == 1].reset_index(drop=True)
df

,Unnamed: 0,file1,file2,same_pattern
0,180,ffcj0179,gkdj0179,1
1,181,ffcj0179,hkdj0179,1
2,182,ffcj0179,jrxx0179,1
3,183,ffcj0179,lzcj0179,1
4,184,ffcj0179,rldj0179,1
...,...,...,...,...
311,1871,gnyx0222,xnyx0222,1
312,1876,hnyx0222,xnyx0222,1
313,1885,lkbn0220,mkbn0220,1
314,1886,lkbn0220,nkbn0220,1


In [30]:
# Caminho do banco de dados SQLite
DB_PATH = "document_comparisons.db"

# Função para remover o prefixo 'patternXX/' do caminho dos arquivos
def clean_filename(filename):
    """
    Remove o caminho do diretório e a extensão '.tif' do nome do arquivo.
    """
    return filename.split("/")[-1].replace(".tif", "")

# Remover o prefixo do caminho dos arquivos
df["file1"] = df["file1"].apply(clean_filename)
df["file2"] = df["file2"].apply(clean_filename)

# Inserir dados na tabela Candidatas
def insert_comparisons(df, modelo="Manual Selection"):
    """
    Insere os dados da comparação na tabela 'Candidatas'.
    """
    # Conectar ao banco de dados
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Verificar e criar a tabela, se não existir
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Candidatas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        doc1_id TEXT NOT NULL,
        doc2_id TEXT NOT NULL,
        similarity_score FLOAT NOT NULL,
        modelo TEXT NOT NULL,
        comparison_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(doc1_id, doc2_id, modelo)
    )
    """)

    # Inserir dados no banco de dados
    for _, row in df.iterrows():
        try:
            cursor.execute("""
            INSERT INTO Candidatas (doc1_id, doc2_id, similarity_score, modelo)
            VALUES (?, ?, ?, ?)
            """, (row["file1"], row["file2"], 1.0, modelo))  # Salva com score 1.0 como padrão
        except sqlite3.IntegrityError:
            print(f"Duplicata ignorada: {row['file1']} - {row['file2']}")

    # Commit e fechar conexão
    conn.commit()
    conn.close()
    print("Dados inseridos com sucesso.")


In [32]:
# Executar a inserção
insert_comparisons(df)

Dados inseridos com sucesso.
